# Phase 5 — Modèle PD (régression logistique)

## Objectif
Entraîner un premier modèle de probabilité de défaut (PD) à partir des features de la Phase 4.
Le modèle choisi est une régression logistique car :
- interprétable (coefficients)
- standard en risque de crédit

## Livrables
- métriques : AUC, accuracy, confusion matrix
- importance/interprétation : coefficients
- exports : fichiers CSV dans `reports/tableau_exports/`


In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report, recall_score, precision_score

In [4]:
#importation of features and target
X = pd.read_csv("../data/processed/X_features.csv")
y = pd.read_csv("../data/processed/y_target.csv").squeeze()

X.shape, y.shape

((250000, 10), (250000,))

In [6]:
#train and test the moel

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.shape, X_test.shape

((200000, 10), (50000, 10))

In [9]:
#Standardisation 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


In [11]:
#modeling
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [29]:
#Predictions (probabilities et classes)
y_proba = model.predict_proba(X_test_scaled)[:,1]
y_pred = model.predict(X_test_scaled)

y_proba[:5], y_pred[:5]

(array([0.23693699, 0.30098   , 0.37173634, 0.20773532, 0.04972834]),
 array([0, 0, 0, 0, 0]))

In [30]:
# Évaluer le modèle (AUC, accuracy, confusion matrix)

auc = roc_auc_score(y_test, y_proba)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

auc

np.float64(0.6999520975007512)

Si je prends un bon client et un mauvais client au hasard,
le modèle donne une probabilité plus élevée au mauvais ≈ 70 % du temps.

In [17]:
acc

0.80018

80 % des prédictions sont correctes.

In [18]:
cm

array([[39729,   292],
       [ 9699,   280]])

lecture :

Prédit----------0 (bon)------------Prédit 1 (défaut)

Vrai 0--------39729 (TN------------292 (FP)

Vrai 1--------9699 (FN)------------280 (TP)

In [19]:
print("AUC:", auc)
print("Accuracy:", acc)
print("\nConfusion Matrix:\n", cm)
print("\nClassification report:\n", classification_report(y_test, y_pred))


AUC: 0.6999520975007512
Accuracy: 0.80018

Confusion Matrix:
 [[39729   292]
 [ 9699   280]]

Classification report:
               precision    recall  f1-score   support

           0       0.80      0.99      0.89     40021
           1       0.49      0.03      0.05      9979

    accuracy                           0.80     50000
   macro avg       0.65      0.51      0.47     50000
weighted avg       0.74      0.80      0.72     50000



### Classe 0 — bons clients
____
#### Interprétation :
Recall 0.99 :
→ le modèle identifie 99 % des bons clients
Precision 0.80 :
→ parmi ceux prédits comme “bons”, 80 % le sont vraiment
#### Très bon pour ne pas refuser de bons clients

___
### Classe 1 — défauts
#### Interprétation :
Recall 0.03 ❌
→ le modèle ne détecte que 3 % des vrais défauts
Precision 0.49
→ quand il prédit “défaut”, il a raison ~1 fois sur 2
C’est le point critique :
#### le modèle rate presque tous les mauvais clients

#### Accuracy (80 %) — trompeuse
accuracy = 0.80

80 % des clients sont bons
Un modèle qui dit “tout le monde est bon” ferait déjà ~80 %
#### Donc accuracy ≠ performance métier ici.


“L’accuracy est élevée, mais le macro F1 est faible, ce qui montre que le modèle ne capte pas correctement la classe minoritaire des défauts.”

In [20]:
coef = pd.DataFrame({
    "feature": X.columns,
    "coef": model.coef_[0]
}).sort_values("coef", ascending=False)

coef.head(15), coef.tail(15)


(            feature      coef
 5           grade_C  0.466224
 6           grade_D  0.443126
 7           grade_E  0.335562
 4           grade_B  0.305248
 2          int_rate  0.217934
 8           grade_F  0.210706
 1       term_months  0.159089
 9           grade_G  0.121523
 0         loan_amnt  0.106348
 3  annual_inc_clean -0.185413,
             feature      coef
 5           grade_C  0.466224
 6           grade_D  0.443126
 7           grade_E  0.335562
 4           grade_B  0.305248
 2          int_rate  0.217934
 8           grade_F  0.210706
 1       term_months  0.159089
 9           grade_G  0.121523
 0         loan_amnt  0.106348
 3  annual_inc_clean -0.185413)

### Explication
récupérer les coefficients du modèle

coef positif → augmente le risque (PD)
coef négatif → diminue le risque

model.coef_[0] contient 1 coefficient par feature
Important : comme on a standardisé, les coefficients sont comparables.

In [22]:
metrics = pd.DataFrame([{
    "auc": auc,
    "accuracy": acc,
    "tn": cm[0,0],
    "fp": cm[0,1],
    "fn": cm[1,0],
    "tp": cm[1,1],
}])

metrics.to_csv("../reports/tableau_exports/05_model_metrics.csv", index=False)
coef.to_csv("../reports/tableau_exports/05_model_coefficients.csv", index=False)

print("Exports OK:")
print("- reports/tableau_exports/05_model_metrics.csv")
print("- reports/tableau_exports/05_model_coefficients.csv")


Exports OK:
- reports/tableau_exports/05_model_metrics.csv
- reports/tableau_exports/05_model_coefficients.csv


### Amélioration du model par une regression logistic avec regularisation (L2)
___

La régression logistique apprend des coefficients (poids) pour chaque variable.
Si certaines variables “poussent trop fort”, le modèle peut :

- sur-apprendre (overfitting)
- devenir instable (coefficients énormes)
- être moins fiable sur des nouvelles données

#### La régularisation empêche les coefficients de devenir trop grands.

In [25]:
model_l2 = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="lbfgs",
    max_iter=1000
)

model_l2.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [31]:
# #Predictions (probabilities et classes) avec la regularisation
y_proba_l2 = model_l2.predict_proba(X_test_scaled)[:,1]
auc = roc_auc_score(y_test, y_proba_l2)
auc


np.float64(0.6999520975007512)

#### L'area under the curve (auc) est similaire 

In [34]:
coef_df = pd.DataFrame({
    "feature": X.columns,
    "coef":model_l2.coef_[0]}).sort_values("coef", ascending = False)
coef_df.head(10)

,feature,coef
5,grade_C,0.466224
6,grade_D,0.443126
7,grade_E,0.335562
4,grade_B,0.305248
2,int_rate,0.217934
8,grade_F,0.210706
1,term_months,0.159089
9,grade_G,0.121523
0,loan_amnt,0.106348
3,annual_inc_clean,-0.185413


In [35]:
coef.head(15)

,feature,coef
5,grade_C,0.466224
6,grade_D,0.443126
7,grade_E,0.335562
4,grade_B,0.305248
2,int_rate,0.217934
8,grade_F,0.210706
1,term_months,0.159089
9,grade_G,0.121523
0,loan_amnt,0.106348
3,annual_inc_clean,-0.185413


In [38]:
threshold = 0.30 #à partir de 30% de probabilité de défaut, je considère le client risqué
y_pred_30 = (y_proba_l2 >= threshold).astype(int)
y_pred_30

array([0, 1, 1, ..., 0, 0, 1])

In [39]:
cm_30 = confusion_matrix(y_test, y_pred_30)
cm_30

array([[34478,  5543],
       [ 6541,  3438]])

In [40]:
cm

array([[39729,   292],
       [ 9699,   280]])

In [49]:
thresholds = [0.2, 0.3, 0.4, 0.5]
rows = []

for t in thresholds:
    pred = (y_proba_l2 >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    rows.append({
        "threshold": t,
        "tn": tn, "fp": fp, "fn": fn, "tp": tp,
        "recall_default": recall_score(y_test, pred),      # rappel = TP/(TP+FN)
        "precision_default": precision_score(y_test, pred) # précision = TP/(TP+FP)
    })

threshold_analysis = pd.DataFrame(rows)
threshold_analysis

,threshold,tn,fp,fn,tp,recall_default,precision_default
0,0.2,24471,15550,3153,6826,0.684036,0.305059
1,0.3,34478,5543,6541,3438,0.344523,0.382808
2,0.4,38259,1762,8581,1398,0.140094,0.442405
3,0.5,39729,292,9699,280,0.028059,0.489510


#### Seuil = 0.20
- TP = 6 826
- FN = 3 153
- Recall ≈ 68 % : détectes 68 % des défauts → très bon / MAIS tu refuses beaucoup de bons clients (FP = 15 550)
- Precision ≈ 30 %  

##### Politique très prudente, mais coûteuse en business.

#### cas d'usage 
- crise économique
- portefeuille très risqué
- priorité absolue à la réduction des défauts

___
#### seuil = 0.30

- TP = 3 438
- FN = 6 541
- Recall ≈ 34 % : détectes 1 défaut sur 3, réduis fortement les faux refus
- Precision ≈ 38 %

##### compromis entre risque et business.
    

___
#### Seuil = 0.40

- TP = 1 398
- FN = 8 581
- Recall ≈ 14 % : rates beaucoup de défauts, protèges le business court terme
- Precision ≈ 44 %

##### Politique agressive, risquée pour la banque.

#### Seuil = 0.50 (par défaut sklearn)

- TP = 280
- FN = 9 699
- Recall ≈ 3 % : rates 97 % des défauts, inutile pour un vrai risque crédit
- Precision ≈ 49 %

##### mauvais choix métier, même si accuracy peut sembler “bonne”.

#### ✅ Seuil = 0.30

Après analyse de l’impact des différents seuils de décision afin de trouver un compromis entre la détection des défauts et la limitation des refus injustifiés. Un seuil de 30 % offre un équilibre cohérent entre risque et performance commerciale.

- ~ 3 438 défauts évités
- ~ 5 543 bons clients refusés
- ~ 6 541 défauts acceptés (reste à gérer via provisions)

In [46]:
coef_df["odds_ratio"] = np.exp(coef_df["coef"])
coef_df = coef_df.sort_values("odds_ratio", ascending=False)

coef_df.head(10)


,feature,coef,odds_ratio
5,grade_C,0.466224,1.593964
6,grade_D,0.443126,1.557569
7,grade_E,0.335562,1.398726
4,grade_B,0.305248,1.356962
2,int_rate,0.217934,1.243505
8,grade_F,0.210706,1.234550
1,term_months,0.159089,1.172442
9,grade_G,0.121523,1.129215
0,loan_amnt,0.106348,1.112209
3,annual_inc_clean,-0.185413,0.830761


Le modèle est cohérent d’un point de vue métier : les grades faibles et les taux élevés augmentent la probabilité de défaut, tandis qu’un revenu plus élevé la réduit. Les coefficients sont interprétables et alignés avec les pratiques de scoring crédit.

## Interprétation des coefficients (Logistic Regression)

Les coefficients du modèle ont été transformés en odds ratios afin de faciliter l’interprétation métier.
___
##### Rappel
odds_ratio > 1 : augmente le risque de défaut
odds_ratio < 1 : diminue le risque de défaut
odds_ratio = 1 : pas d’effet
___
#### 🔴 Facteurs qui augmentent le risque de défaut

grade_C (OR = 1.59)
→ Un prêt de grade C a environ +59 % de risque de défaut par rapport au grade de référence (grade A).

grade_D (OR = 1.56)
→ Le risque de défaut augmente d’environ +56 %.

grade_E (OR = 1.40)
→ Le risque est +40 % plus élevé.

grade_B (OR = 1.36)
→ Même un grade relativement correct présente un sur-risque modéré.

##### Conclusion :
Le grade de crédit est la variable la plus discriminante du modèle, ce qui est cohérent avec la logique métier de LendingClub.

#### 🔴 Autres variables de risque

int_rate (OR = 1.24)
→ Une hausse du taux d’intérêt est associée à une augmentation du risque de défaut.

(Logique : taux élevé = client plus risqué)

term_months (OR = 1.17)
→ Les prêts plus longs (ex : 60 mois) sont plus risqués que les prêts courts.

loan_amnt (OR = 1.11)
→ Des montants de prêt plus élevés sont associés à un léger sur-risque.

#### 🟢 Facteur qui réduit le risque de défaut
annual_inc_clean (OR = 0.83)
→ Un revenu annuel plus élevé réduit le risque de défaut d’environ 17 %.

Plus l’emprunteur gagne, plus il est capable d’honorer ses échéances.

___

### Conclusion 

Le modèle identifie le grade de crédit, le taux d’intérêt et la durée du prêt comme les principaux facteurs explicatifs du défaut.
À l’inverse, un revenu annuel élevé agit comme un facteur protecteur.
Ces résultats sont cohérents avec les principes classiques du credit risk management.


In [47]:
coef_df.to_csv(
    "../reports/tableau_exports/05_model_coefficients_interpreted.csv",
    index=False
)


In [48]:


decision_threshold = pd.DataFrame([{
    "model": "logistic_regression_l2",
    "decision_threshold": 0.30,
    "business_objective": "balance_risk_vs_business",
    "justification": "Compromise between default recall and false positives",
    "expected_recall_default": 0.34,
    "expected_precision_default": 0.38
}])

decision_threshold.to_csv(
    "../reports/tableau_exports/05_decision_threshold.csv",
    index=False
)

decision_threshold


,model,decision_threshold,business_objective,justification,expected_recall_default,expected_precision_default
0,logistic_regression_l2,0.3,balance_risk_vs_business,Compromise between default recall and false po...,0.34,0.38


In [50]:
threshold_analysis.to_csv(
    "../reports/tableau_exports/05_threshold_analysis.csv",
    index=False
)

print("Export OK : reports/tableau_exports/05_threshold_analysis.csv")


Export OK : reports/tableau_exports/05_threshold_analysis.csv
